# M2 Bending Mode Test

This Jupyter notebook is to run the bending mode test of M2.
Each axial actuator will be issued with an additional force according to specific bending mode (there are 69 in total).
The target is to understand the mirror supporting system can hold the specific shape based on bending mode or not.

IMPORTANT NOTE: the bending mode dataset starts from index 0.

## Import Modules

In [1]:
import asyncio
import yaml
import numpy as np
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK
from lsst.ts.ofc.utils import get_config_dir as get_config_dir_ofc

## Declaration of User-defined Functions

In [70]:
def get_bending_mode():
    """Get the bending mode.

    Returns
    -------
    bending_mode : `numpy.ndarray`
        Bending mode.
    """
    bending_mode_file = get_config_dir_ofc() / "M2" / "M2_1um_72_force.yaml"
    with open(bending_mode_file, "r") as yaml_file:
        bending_mode = np.array(yaml.safe_load(yaml_file))

    
    return bending_mode

In [3]:
def get_bending_mode_forces(bending_mode, idx_bending_mode, amplitude):
    """Plot the forces of bending mode.
    
    Parameters
    ----------
    bending_mode : `numpy.ndarray`
        Bending mode data.
    idx_bending_mode : `int`
        Index of bending mode (1-20).
    amplitude : `float`
        Amplitude of bending mode in um.
        
    Returns
    -------
    `numpy.ndarray`
        Actuator forces in Newton.
    """

    # Column 0-2 are the actuator ID, x and y position of actuator
    return amplitude * bending_mode[:, 2+idx_bending_mode]

In [4]:
async def apply_force_cycle_axial(csc, forces, time_forces=5, time_reset=5):
    """Apply the force cycle to axial actuators. The cycle will be positive force, clear,
    negative force, clear.

    Parameters
    ----------
    csc : `lsst.ts.salobj.remote.Remote`
        Remote object of the M2 CSC.
    forces : `numpy.ndarray`
        Axial forces to apply (Newton).
    time_forces : `float`, optional
        Time to apply the forces in second. (the default is 5.0) 
    time_reset : `float`, optional
        Time to reset the forces in second. (the default is 5.0)
    """
    
    # Do the positive direction first
    print(f"Apply the force: {forces} N.")
    await csc.cmd_applyForces.set_start(axial=forces.tolist())
    await asyncio.sleep(time_forces)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(time_reset)

    # Do the Negative direction
    forces_negative = -forces

    print(f"Apply the force: {forces_negative} N.")
    await csc.cmd_applyForces.set_start(axial=forces_negative.tolist())
    await asyncio.sleep(time_forces)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(time_reset)

## Prepare the M2 CSC and put to Enabled state

In [5]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [73]:
# get status

state = m2.evt_summaryState.get()
print(state)

private_revCode: 0ebebdcc, private_sndStamp: 1688429850.9569166, private_rcvStamp: 1688432161.0048306, private_seqNum: 30, private_identity: MTM2, private_origin: 59180, summaryState: 2


In [ ]:
# Standby  -->  Disable
await m2.cmd_start.set_start(timeout=60)

In [ ]:
# Disable  -->  Enabled
await m2.cmd_enable.set_start(timeout=200)

In [ ]:
# Enabled  -->  Disable
await m2.cmd_disable.set_start(timeout=30)

In [ ]:
#Disable  -->  Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
#Fault --> Standby
await m2.cmd_standby.set_start(timeout=30)

## Apply one Bending Modes to fine tune the amplitude/scaling factor

The fine tuning of the bending mode scaling factor is done for groups of 10 bending modes at the time. This step can be skipped if the scaling factors are already defined.

In [7]:
bending_mode = get_bending_mode()

In [75]:
# bending modes vector indexes
decina = 0 #Select the tens-order of the bending mode, from 0 to 6  
idx = 0 #Select the bending mode inside the tens-order, from 0 to 9

# genereate a vector of 10 elements to be populated with the ascending bending mode order 
idx_bending_mode = [0] * 10
for idx in range(10):
    idx_bending_mode[idx] = idx + decina*10
    
print(idx_bending_mode)
# bending mode scaling factors, to be fine tuned, applied force shall be around 10 N to get 500 nm RMS surface deformation
amplitude = np.array([1, 0.1, 0.1, 0.05, 0.025, 0.01, 0.01])


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [61]:
# bending mode number from 0 to 9

print(f"Bending mode is {idx_bending_mode[idx]} and amplitude is {amplitude[decina]}")
forces = get_bending_mode_forces(bending_mode, idx_bending_mode[idx]+1, amplitude[0])

# apply single bending mode
await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)

Bending mode is 0 and amplitude is 10.0
Apply the force: [ 5.095989   5.131827   4.276799   2.677288   0.6109366 -1.560001
 -3.467116  -4.773617  -5.248473  -4.809246  -3.53598   -1.664711
  0.4968142  2.571184   4.198307   5.095987   5.131827   4.276799
  2.677289   0.6109379 -1.560001  -3.467114  -4.773617  -5.248472
 -4.809247  -3.535979  -1.664713   0.4968123  2.571182   4.198308
  3.77689    3.199174   1.7611    -0.1525739 -2.028289  -3.360795
 -3.787747  -3.194907  -1.748602   0.1616077  2.026647   3.347492
  3.776889   3.199172   1.7611    -0.1525752 -2.028287  -3.360795
 -3.787749  -3.194908  -1.748602   0.1616072  2.026646   3.347493
  2.533815   2.309553   1.003845  -0.7756682 -2.19378   -2.581402
 -1.758154  -0.1157655  1.577566   2.533812   2.309553   1.003844
 -0.7756679 -2.193778  -2.581403  -1.758154  -0.1157658  1.577566 ] N.
Reset the force.
Apply the force: [-5.095989  -5.131827  -4.276799  -2.677288  -0.6109366  1.560001
  3.467116   4.773617   5.248473   4.809246   

## Looping over multiple bending modes 
Loop over multiple bending modes with positive and negative signs at groups of 10 bending modes. 

In [ ]:
bending_mode = get_bending_mode()

In [87]:
# array definition

# bending modes scaling factors
amplitude = np.array([1, 0.1, 0.1, 0.05, 0.025, 0.01, 0.01])

In [104]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")


# bending modes from 1 to 10

for idx in range(0,10):
    print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[0]}")
    forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[0])
    # apply single +/- bending mode 
    await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

UTC time to is 2023-07-04 03:34:00.488664 now.
Status of advancement: Bending mode 1, mode amplitude 1.0
Apply the force: [ 0.5095989   0.5131827   0.4276799   0.2677288   0.06109366 -0.1560001
 -0.3467116  -0.4773617  -0.5248473  -0.4809246  -0.353598   -0.1664711
  0.04968142  0.2571184   0.4198307   0.5095987   0.5131827   0.4276799
  0.2677289   0.06109379 -0.1560001  -0.3467114  -0.4773617  -0.5248472
 -0.4809247  -0.3535979  -0.1664713   0.04968123  0.2571182   0.4198308
  0.377689    0.3199174   0.17611    -0.01525739 -0.2028289  -0.3360795
 -0.3787747  -0.3194907  -0.1748602   0.01616077  0.2026647   0.3347492
  0.3776889   0.3199172   0.17611    -0.01525752 -0.2028287  -0.3360795
 -0.3787749  -0.3194908  -0.1748602   0.01616072  0.2026646   0.3347493
  0.2533815   0.2309553   0.1003845  -0.07756682 -0.219378   -0.2581402
 -0.1758154  -0.01157655  0.1577566   0.2533812   0.2309553   0.1003844
 -0.07756679 -0.2193778  -0.2581403  -0.1758154  -0.01157658  0.1577566 ] N.
Reset the

In [105]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending modes from 11 to 20

for idx in range(10,20):
        print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[1]}")
        forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[1])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

UTC time to is 2023-07-04 03:37:48.972930 now.
Status of advancement: Bending mode 11, mode amplitude 0.1
Apply the force: [-0.3327048   0.2830594   0.8615695   1.283361    1.471451    1.415027
  1.118711    0.6309827   0.02830515 -0.5793967  -1.082316   -1.401762
 -1.492558   -1.311639   -0.8920552  -0.332708    0.2830446   0.8615571
  1.283358    1.471455    1.415027    1.118706    0.6309722   0.02829421
 -0.5793993  -1.082317   -1.401754   -1.492555   -1.311649   -0.8920697
 -0.04236923 -0.5580639  -0.9176477  -1.028243   -0.8689312  -0.4837569
  0.03007839  0.5398218   0.9112935   1.041813    0.8875647   0.4884135
 -0.04236735 -0.5580616  -0.9176403  -1.028229   -0.8689234  -0.4837553
  0.03007764  0.5398284   0.9113033   1.041819    0.8875647   0.4884113
  0.6666343  -1.374993   -2.753147   -2.83534    -1.61912     0.3501842
  2.168703    2.99411     2.402967    0.6666341  -1.374989   -2.753142
 -2.835336   -1.619124    0.3501802   2.168703    2.994112    2.402966  ] N.
Reset the 

In [106]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending modes from 21 to 30

for idx in range(20,30):
        print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[2]}")
        forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[2])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

UTC time to is 2023-07-04 03:41:38.494862 now.
Status of advancement: Bending mode 21, mode amplitude 0.1
Apply the force: [-11.51466     6.03895    13.01705    -3.499795  -13.63565     0.611601
  13.77873     2.282677  -13.40139    -4.903563   12.12618     7.739787
 -10.73429    -9.901479    8.731928   11.51458    -6.03896   -13.01685
   3.499703   13.63563    -0.6115677 -13.77865    -2.282755   13.40122
   4.903726  -12.12607    -7.739853   10.73421     9.901577   -8.73195
  -0.5797576   6.425042   -2.788737   -4.971881    5.327498    2.250777
  -6.498655    1.156846    5.907211   -4.174189   -3.709969    6.12868
   0.5797988  -6.425075    2.788589    4.97188    -5.327457   -2.250804
   6.498603   -1.156834   -5.907229    4.174129    3.71001    -6.128633
  -1.994619    2.40998    -1.634369    0.1060815   1.467408   -2.373055
   2.100623   -0.9427727  -0.7386997   1.994665   -2.410051    1.634326
  -0.1059651  -1.467325    2.373053   -2.100673    0.9427353   0.7387473] N.
Reset the fo

In [107]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending modes from 31 to 40

for idx in range(30,40):
        print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[3]}")
        forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[3])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

UTC time to is 2023-07-04 03:45:26.045128 now.
Status of advancement: Bending mode 31, mode amplitude 0.05
Apply the force: [-5.9593800e-03  1.7402200e+01 -1.0795070e+01 -1.0804935e+01
  1.7400110e+01  6.1663100e-03 -1.7403350e+01  1.0795075e+01
  1.0805340e+01 -1.7398755e+01 -5.4625000e-03  1.7402165e+01
 -1.0796215e+01 -1.0804840e+01  1.7399615e+01  5.3416150e-03
 -1.7403415e+01  1.0795085e+01  1.0804955e+01 -1.7398385e+01
 -4.8101675e-03  1.7401750e+01 -1.0794885e+01 -1.0804780e+01
  1.7399650e+01  5.6140550e-03 -1.7403520e+01  1.0794625e+01
  1.0804935e+01 -1.7398385e+01  6.7262250e+00 -2.5311355e+00
 -2.5375055e+00  6.7272650e+00 -6.7258300e+00  2.5317430e+00
  2.5356395e+00 -6.7292250e+00  6.7267050e+00 -2.5300695e+00
 -2.5360885e+00  6.7285200e+00 -6.7247550e+00  2.5324120e+00
  2.5353485e+00 -6.7299800e+00  6.7255850e+00 -2.5312175e+00
 -2.5372125e+00  6.7281200e+00 -6.7246450e+00  2.5327710e+00
  2.5363280e+00 -6.7288500e+00 -5.4141800e-04 -2.4380790e-01
 -2.4775570e-01  4.741

In [108]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending modes from 41 to 50

for idx in range(40,50):
        print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[4]}")
        forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[4])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

UTC time to is 2023-07-04 03:49:18.095496 now.
Status of advancement: Bending mode 41, mode amplitude 0.025
Apply the force: [  4.8838175    3.215165    -0.18820433  -3.0355025   -4.468245
  -2.809        1.10717225   3.6591225    3.9320725    1.841524
  -2.07483375  -4.322625    -3.4703775   -0.8971595    2.62695
   4.88432      3.214575    -0.1878642   -3.035675    -4.468725
  -2.8088375    1.10698975   3.6590275    3.9322825    1.84163275
  -2.075246    -4.3220025   -3.470985    -0.89662575   2.6272175
 -13.5375325   -1.5139685   11.960075    13.015505     1.8823275
 -11.65401    -12.94191     -0.65611625  11.6550625   13.1684075
   0.98167    -12.359255   -13.538015    -1.51377325  11.9606175
  13.0159525    1.88280225 -11.654295   -12.94209     -0.65608025
  11.65524     13.168165     0.98134525 -12.359885    13.1244375
   3.6625875  -11.8072925   -7.5500425    8.849635    10.32746
  -5.5740375  -12.5125575    1.4797535   13.1245       3.6626025
 -11.80761     -7.5506425    8.8498

In [109]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending modes from 51 to 60

for idx in range(50,60):
        print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[5]}")
        forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[5])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

UTC time to is 2023-07-04 03:53:01.185503 now.
Status of advancement: Bending mode 51, mode amplitude 0.01
Apply the force: [11.83733  -8.811439  2.866388  2.861692 -8.808932 11.83684  -8.810828
  2.865623  2.86271  -8.810179 11.83829  -8.812118  2.86691   2.861905
 -8.809912 11.83811  -8.811824  2.866478  2.862172 -8.810367 11.83889
 -8.813252  2.868066  2.860992 -8.809144 11.83751  -8.811742  2.866709
  2.861828 -8.8094   -1.536623  1.563663  1.564426 -1.534312 -1.536562
  1.563737  1.564217 -1.533959 -1.537269  1.563702  1.564216 -1.533775
 -1.53709   1.563595  1.564509 -1.533738 -1.537003  1.563732  1.56366
 -1.533121 -1.537252  1.563666  1.564167 -1.53385   2.2655   -1.132868
 -1.134795  2.265563 -1.132666 -1.135223  2.26619  -1.132933 -1.134803
  2.265514 -1.132476 -1.135227  2.265602 -1.132457 -1.135057  2.265601
 -1.132655 -1.134857] N.
Reset the force.
Apply the force: [-11.83733    8.811439  -2.866388  -2.861692   8.808932 -11.83684
   8.810828  -2.865623  -2.86271    8.81017

In [110]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending modes from 61 to 69

for idx in range(60,68):
        print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[6]}")
        forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[6])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

UTC time to is 2023-07-04 03:56:50.104820 now.
Status of advancement: Bending mode 61, mode amplitude 0.01
Apply the force: [  7.011475  -10.51239    10.86623   -11.43066    14.32452   -16.56907
  15.15884   -12.5604     12.14604   -12.36236     9.559994   -4.648655
   1.697193   -0.7189374  -1.957436    7.005379  -10.50738    10.86115
 -11.42603    14.32048   -16.56608    15.1562    -12.5595     12.14615
 -12.36296     9.558907   -4.646436    1.694323   -0.7149527  -1.963228
   3.331785    2.686922   -6.000093    3.432208    2.53591    -6.028079
   3.597999    2.424726   -5.869003    3.341858    2.401085   -5.857838
   3.332856    2.68685    -5.999108    3.430446    2.539303   -6.029304
   3.598602    2.425454   -5.867479    3.338663    2.404372   -5.858741
   3.054108   -5.144196    6.544821   -7.215955    6.99005    -5.962862
   4.163999   -1.846426   -0.5815668   3.052992   -5.144652    6.545306
  -7.216756    6.989447   -5.962183    4.161989   -1.844415   -0.5834918] N.
Reset the 

## Bending modes repeatability
Apply for 3 times the same bending mode with positive and negative sign

In [111]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending mode 15 repeatability x3 times

for repetition in range(0,3):
        print(f"Status of advancement: Bending mode {15}, mode amplitude {amplitude[1]}")
        forces = get_bending_mode_forces(bending_mode, 15, amplitude[1])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

UTC time to is 2023-07-04 03:59:50.882524 now.
Status of advancement: Bending mode 15, mode amplitude 0.1
Apply the force: [-3.251299 -2.644937 -1.021811  1.012735  2.635241  3.241197  2.635161
  1.012473 -1.022111 -2.645093 -3.251309 -2.644941 -1.021815  1.012714
  2.635274  3.241186  2.635167  1.012452 -1.022109 -2.645081 -3.251316
 -2.644942 -1.021821  1.012726  2.635264  3.241212  2.635146  1.012461
 -1.022101 -2.645107  2.662804  1.104778 -1.104255 -2.662459 -2.662407
 -1.104152  1.104867  2.662844  2.662814  1.104806 -1.10425  -2.662463
 -2.662417 -1.104145  1.104885  2.662847  2.662812  1.10479  -1.104242
 -2.662463 -2.662405 -1.104161  1.104878  2.662854  6.871742  3.447065
 -3.432108 -6.856374 -3.43155   3.447647  6.87174   3.447062 -3.432119
 -6.856381 -3.431541  3.447635  6.871747  3.447071 -3.432139 -6.856386
 -3.431528  3.447639] N.
Reset the force.
Apply the force: [ 3.251299  2.644937  1.021811 -1.012735 -2.635241 -3.241197 -2.635161
 -1.012473  1.022111  2.645093  3.251